# Starbucks Customer Segmentation

## I. Outline

## II. Introduction

### II.I Business Understanding

### II.II Data Understanding

## III. Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## IV. Data Loading

In [32]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## V. Data Cleaning

In [33]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [34]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [35]:
portfolio.shape

(10, 6)

In [36]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [37]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [38]:
profile.shape

(17000, 5)

In [39]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [40]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [41]:
transcript.shape

(306534, 4)

### V.I Data Transform

In [42]:
# One hot encode channels
channels = []
for channel in portfolio['channels']:
  channels.extend(channel)

channels = set(channels)

for channel in channels:
  portfolio[channel] = portfolio['channels'].apply(lambda x: 1 if channel in x else 0)

portfolio.drop(columns=['channels'], inplace=True)
portfolio

,reward,difficulty,duration,offer_type,id,mobile,email,web,social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,0,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,1,0
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,0,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0


In [43]:
# extract keys in value column
keys = []
for value in transcript['value']:
  keys.extend(value.keys())

value_keys = set(keys)

# extract value column in transcript
for key in value_keys:
  transcript[key] = transcript['value'].apply(lambda x: x[key] if key in x.keys() else 0)
  

# combine offer id and offer_id column
transcript['offer_id'] = np.where(transcript['offer id'] != 0, transcript['offer id'], transcript['offer_id'])

#drop value and offer id column in transcript
transcript.drop(columns=['value','offer id'], inplace=True)

In [52]:
# Combine the data
df_customer = profile.merge(transcript, left_on='id', right_on='person', how="inner")
print(df_customer.shape)
df = df_customer.merge(portfolio, left_on='offer_id', right_on='id', how="left")
print(df.shape)

(306534, 11)
(306534, 20)


In [53]:
df.head()

,gender,age,id_x,became_member_on,income,person,event,time,amount,reward_x,offer_id,reward_y,difficulty,duration,offer_type,id_y,mobile,email,web,social
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,68be06ca386d4c31939f3a4f0e3dd783,offer received,168,0.00,0,2906b810c7d4411798c6938adc9daaa5,2.0,10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,1.0,1.0,1.0,0.0
1,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,68be06ca386d4c31939f3a4f0e3dd783,offer viewed,216,0.00,0,2906b810c7d4411798c6938adc9daaa5,2.0,10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,1.0,1.0,1.0,0.0
2,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,68be06ca386d4c31939f3a4f0e3dd783,offer received,336,0.00,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,20.0,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,1.0,1.0,0.0
3,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,68be06ca386d4c31939f3a4f0e3dd783,offer viewed,348,0.00,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,20.0,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,1.0,1.0,0.0
4,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,68be06ca386d4c31939f3a4f0e3dd783,transaction,360,0.35,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df['reward'] = np.where(df['reward_x'] != 0, df['reward_x'], df['reward_y'])
df.drop(columns=['person','offer_id','id_y','reward_x','reward_y'], inplace=True)
df.rename(columns={'id_x':'id'}, inplace=True)
df.head()

,gender,age,id,became_member_on,income,event,time,amount,difficulty,duration,offer_type,mobile,email,web,social,reward
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer received,168,0.00,10.0,7.0,discount,1.0,1.0,1.0,0.0,2.0
1,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer viewed,216,0.00,10.0,7.0,discount,1.0,1.0,1.0,0.0,2.0
2,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer received,336,0.00,20.0,10.0,discount,0.0,1.0,1.0,0.0,5.0
3,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer viewed,348,0.00,20.0,10.0,discount,0.0,1.0,1.0,0.0,5.0
4,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,transaction,360,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df.shape

(306534, 16)

### V.II Handling Missing Value

In [56]:
df.isna().sum()

gender               33772
age                      0
id                       0
became_member_on         0
income               33772
event                    0
time                     0
amount                   0
difficulty          138953
duration            138953
offer_type          138953
mobile              138953
email               138953
web                 138953
social              138953
reward              138953
dtype: int64

In [49]:
df[df['gender'].isna() & df['income'].isna()].shape

(18776, 16)

## VI. Exploratory Data Analysis

## VII. Data Preprocessing

## VIII. Modeling

## IX. Evaluation

## X. Model Tuning

## XI. Conclusion